In [1]:
import tweepy
import pandas as pd
import json
from io import StringIO

  
# assign the values accordingly 
consumer_key = "Bitte selbst ausfüllen!" 
consumer_secret = "Bitte selbst ausfüllen!" 
access_token = "Bitte selbst ausfüllen!" 
access_token_secret = "Bitte selbst ausfüllen!"

In [3]:
#https://stackoverflow.com/questions/44581647/retrieving-a-list-of-tweets-using-tweet-id-in-tweepy
def lookup_tweets(tweet_IDs, api):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range(int((tweet_count / 100) + 1)):
            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)
            full_tweets.extend(
                api.statuses_lookup(id_=tweet_IDs[i * 100:end_loc], tweet_mode="extended")
            )
        return full_tweets
    except tweepy.TweepError:
        print('Something went wrong, quitting...')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# do whatever it is to get ids to where df.TweetID is - the list of all IDs to look up
df = pd.read_csv("data/500000tweets.tsv", sep='\t')
good_tweet_ids = [i for i in df.TweetID] #tweet ids to look up 

results = lookup_tweets(good_tweet_ids, api) #apply function

Rate limit reached. Sleeping for: 310
Rate limit reached. Sleeping for: 318


In [4]:
temp = json.dumps([status._json for status in results]) #create JSON
newdf = pd.read_json(StringIO(temp))
#newdf = pd.read_json(temp, orient='records')
full = pd.merge(df, newdf, left_on='TweetID', right_on='id', how='right').drop('id', axis=1)

selectionString = ['TweetID', 'Datetime', 'Followers', 'favorite_count', 'retweet_count', 'Sentiment', 'full_text']
full['full_text'] = full['full_text'].str.replace('\n',' ')
full['full_text'] = full['full_text'].str.replace('\r',' ')
full[selectionString].to_csv("data/500000TweetsHydrated.tsv",index = False,sep='\t',line_terminator='\r\n')